## 🚀 Day2-02：Agent Tools 模式与最佳实践

5 天 AI Agents 强化课程的第二天，我们将学习如何使用工具来扩展智能体的功能。

上小节我们学习了如何将自定义 Python 函数作为工具添加到您的智能体中。
现在，我们将更进一步：使用外部 MCP 服务并处理长时间运行的操作。

你讲学习到内容：

- ✅ 连接外部 MCP 服务器
- ✅ 实现可暂停代理执行以等待外部输入的长时运行操作
- ✅ 构建可恢复的工作流，在对话中断时保持状态
- ✅ 了解何时以及如何使用这些模式

本次课程直播回放请到 youtube 观看: [youtube](https://www.youtube.com/playlist?list=PLqFaTIg4myu9r7uRoNfbJhHUbLp-1t1YE)

入门步骤：

1.vscode 安装 jupyter 插件

# Section 1：设置环境

##### 1.1 配置你的 Gemini API密钥

In [ ]:
import os
from pathlib import Path

# 读取项目根目录的 .env 文件
env_file = Path.cwd().parent / '.env'

if env_file.exists():
    for line in env_file.read_text().splitlines():
        if line.startswith('GOOGLE_API_KEY='):
            os.environ["GOOGLE_API_KEY"] = line.split('=', 1)[1].strip()
            print("✅ Gemini API key setup complete.")
            break
else:
    print(f"⚠️ Please create .env file at: {env_file}")


✅ Gemini API key setup complete.


##### 1.2 导入 ADK 组件

基于 Google 的 Agent Development Kit (ADK) 框架来构建 Agent。ADK 提供了一组工具和库，用于构建和运行 Agent。

In [1]:
import uuid
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


##### 1.3：配置重试选项

当你在使用LLM时，可能会遇到瞬时错误，如速率限制或临时服务不可用。重试选项通过指数退避自动重试请求来处理这些故障。

In [ ]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

# 🤖 Section 2: MCP（Model Context Protocol）